In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [10]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

In [11]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['NT_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Apr_list)

60480

In [13]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 2520)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2516)

In [15]:
x[:,:2]

matrix([[ 3604.00833803,  3536.30874654],
        [ 3400.32703429,  3417.55878093],
        [ 4470.60008165,  4441.6109453 ],
        [ 4417.44600785,  4465.18797617],
        [ 4391.55679787,  4491.13579477],
        [ 4117.71521812,  4204.62134947],
        [ 3680.37857224,  3713.228648  ],
        [ 3633.01366936,  3668.08246955],
        [ 4896.37125813,  4870.37010637],
        [ 4851.34775136,  4758.30905649],
        [ 4079.43860397,  4353.24558389],
        [ 3678.72287993,  3672.88188334],
        [ 3432.2902516 ,  3442.63109993],
        [ 3967.53389257,  4007.28308923],
        [ 3574.64046677,  3624.79061037],
        [ 2796.52083056,  2892.2839259 ],
        [ 3604.96228945,  3671.93464521],
        [ 3528.22256334,  3604.79440673],
        [ 4160.25380027,  4052.55833892],
        [ 3889.85733175,  3938.33963816],
        [ 4157.76603586,  4111.43558856],
        [ 3978.22876562,  3910.57949976],
        [ 3585.50044911,  3719.179315  ],
        [ 1940.70685733,  3690.877

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [4e+00, 2e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.38834704e+07  0.00000000e+00  5.66e+03 1.43e+03  1.00e+06     0s
   1  -2.11385779e+07 -1.20661915e+07  1.15e+03 2.89e+02  2.69e+05     0s
   2  -1.61680771e+07 -1.74273125e+07  7.44e+01 1.88e+01  5.12e+04     0s
   3  -1.59695862e+07 -1.61279396e+07  7.85e-01 1.98e-01  3.1

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_NT.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.2364132072989051e-11,
 0.33570958379194593,
 1.3606802572166925e-11,
 0.0007032023483219685,
 1355.406675883347,
 5745.9543179910825,
 4.10611121434664e-09,
 5.588410436242915e-05,
 2.679484309299023e-10,
 3490.9115288666417,
 2.679484165837996e-10,
 2.6794830522933364e-10,
 2.67948431173823e-10,
 2.6948496941255556e-10,
 1641.7848735686423,
 3124.988697005906,
 582.2128710325591,
 8.051661212765083,
 0.0004780897536740686,
 2.831282611640347,
 839.578240335156,
 2295.5206426948025,
 462.22504691698634,
 894.8213979816585,
 0.11763126730734763,
 2001.7896051754801,
 0.00021795605975342492,
 3250.504074408173,
 1.0136206509668,
 0.9358081541086882,
 2.8310892972036044,
 259.1220268611682,
 1086.5466369645276,
 0.00021448346737937577,
 0.00021655252809675606,
 3.6684265246009264,
 13.619583382851063,
 3.8920688392609626,
 2693.153020877706,
 0.43958490385288584,
 664.9149425730399,
 434.3343596423644,
 3405.7725367407247,
 2.916975925043605,
 5.303796224451958,
 3.1226653447772606e-06